In [1]:
from src.rag.components.retriever import HybridRetriever

/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
embedding_model_id = "dunzhang/stella_en_400M_v5"

In [3]:
from pathlib import Path

In [4]:
model_path = Path.cwd().joinpath("models")

In [5]:
embedding_model_path = model_path.joinpath(embedding_model_id ).__str__()

In [6]:
cross_encoder_kwargs = {
    "model_name": embedding_model_path,
    "trust_remote_code": True,
    "local_files_only": True,
    "config_kwargs": {"use_memory_efficient_attention": False, "unpad_inputs": False},
    "device": "cpu"
}

In [7]:

transformer_kwargs  = { "model_name_or_path": embedding_model_path,
    "trust_remote_code": True,
    "device": "cpu",
    "config_kwargs": {"use_memory_efficient_attention": False,
                   "unpad_inputs": False},
    "cache_folder": model_path}

In [8]:
spacy_model_id = "en_core_web_sm"

In [9]:
retriever = HybridRetriever(
    cross_encoder_kwargs=cross_encoder_kwargs,
    spacy_model=spacy_model_id,
    sentence_transformer_kwargs=transformer_kwargs,
    language="english"
)

Some weights of the model checkpoint at /Users/esp.py/Projects/Personal/end-to-end-rag/models/dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of NewForSequenceClassification were not initialized from the model checkpoint at /Users/esp.py/Projects/Personal/end-to-end-rag/models/dunzhang/stella_en_400M_v5 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predi

In [29]:
query = "How can I speed up the mortgage completion process?"


## "How can I speed up the mortgage completion process?"


#"How does an interest only mortgage work?"

# "How do you work out your LTV ratio?"

In [30]:
semantic_search_results = retriever.semantic_search(query=query, limit=10)

In [31]:
keywords = retriever.perform_keyword_extraction(query)

In [32]:
keywords

'"mortgage completion process"'

In [33]:
keywords_search_results = retriever.keyword_search(keywords=keywords, limit=10)

We can add a reranker but we decided to keep it simple for now.

In [34]:
for item in semantic_search_results:
    print(item.content)
    print(10 * "-")

Knowing how to remortgage your home quickly can speed up the process. While there is no guaranteed fast-track, there are a few things you can do.
----------
How long will my mortgage application take?                                  expandable section
----------
What could delay my mortgage application?                                   expandable section
----------
Worried about the paperwork and process behind a mortgage? Find out how to apply for one here.
----------
Fancy a refresher on the mortgage process? You'll find everything you need to know here.
----------
How could you increase your chances of being accepted for a mortgage?                                  expandable section
----------
Being in a strong financial position can affect how quick the process is. If you are trying to borrow more money, and if there’s any issue with your income or credit history, this could cause delays.
----------
Save yourself time by getting a mortgage Agreement in Principle from your lender

For some reason the encoder is taking long to return.

In [35]:
for item in keywords_search_results:
    print(item.content)
    print(10 * "-")

As long as one person applying has never owned a property before, you can apply for a First Time Buyer Mortgage with the Halifax. You’ll need a deposit of at least 5% of the property’s value, but if you can afford more than 5%, you can often get a lower initial interest rate. We’re supporting the Government’s mortgage guarantee scheme, if you only have a deposit of at least 5% but less than 10%. The scheme is expected to accept applications until 30 June 2025, however, it may be withdrawn earlier. You can apply for a mortgage under the scheme by following our usual application process. If you’re planning to put down a deposit between 5% and 10%, to qualify for the Mortgage Guarantee Scheme you’ll need to be:  A first-time buyer Thinking about borrowing less than £570,000 Buying a property that isn’t a new build flat Buying a property that isn’t Shared Ownership, Shared Equity, Right to Buy or buy to let Thinking about getting a repayment mortgage and not interest only Lending is subjec

In [36]:
# scores = retriever.cross_encoder.predict(
#    [(query, item.content) for item in keywords_search_results + semantic_search_results])

### Generator

In [37]:
from src.rag.components.generator import  LLamaCppGeneratorComponent

In [38]:
prompt = "you are a helpful mortgage advisor"

In [39]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

In [40]:
llama_cpp_generator = LLamaCppGeneratorComponent(
    api_url="http://127.0.0.1:8001",
    model_name=model_name,
    prompt=prompt
)

In [41]:
llama_cpp_generator._ping_api()

True

In [42]:
documents = [doc.content for doc in semantic_search_results +
             keywords_search_results]

In [43]:
documents

['Knowing how to remortgage your home quickly can speed up the process. While there is no guaranteed fast-track, there are a few things you can do.',
 'How long will my mortgage application take?                                  expandable section',
 'What could delay my mortgage application?                                   expandable section',
 'Worried about the paperwork and process behind a mortgage? Find out how to apply for one here.',
 "Fancy a refresher on the mortgage process? You'll find everything you need to know here.",
 'How could you increase your chances of being accepted for a mortgage?                                  expandable section',
 'Being in a strong financial position can affect how quick the process is. If you are trying to borrow more money, and if there’s any issue with your income or credit history, this could cause delays.',
 'Save yourself time by getting a mortgage Agreement in Principle from your lender before you make an offer. This can speed up th

In [44]:
template_input = {
    "documents": documents,
    "question": query
}

In [45]:
llama_cpp_generator.generate_chat_input(template_values=template_input)

[{'role': 'system', 'content': 'you are a helpful mortgage advisor'},
 {'role': 'user',
  'content': "\n            DOCUMENTS:\n            \n             - Knowing how to remortgage your home quickly can speed up the process. While there is no guaranteed fast-track, there are a few things you can do. \n\n            \n             - How long will my mortgage application take?                                  expandable section \n\n            \n             - What could delay my mortgage application?                                   expandable section \n\n            \n             - Worried about the paperwork and process behind a mortgage? Find out how to apply for one here. \n\n            \n             - Fancy a refresher on the mortgage process? You'll find everything you need to know here. \n\n            \n             - How could you increase your chances of being accepted for a mortgage?                                  expandable section \n\n            \n             - Be

In [46]:
response = llama_cpp_generator.run(template_values=template_input)

In [47]:
print(response)

To speed up the mortgage completion process, follow these steps:

1. Return any requested documentation for your mortgage as soon as possible.
2. Work closely with your conveyancer to understand timings and next steps in the process – such as local authority search turnaround times.
3. Make sure all parties are working towards the same completion date and be aware of any chains you may be in which may impact this.
4. Consider any other Third Parties you’ll need to contact and get quotes from (such as removal firms), and make sure they are aware of the completion date you are aiming for.
5. Ensure that all necessary paperwork is submitted promptly, including any required certificates or approvals.

By following these steps, you can help streamline the mortgage process and potentially expedite the completion of your mortgage application.
